# Supervised Learning In-Depth: Model selection

Previously we saw the main API shared by objects in scikit-learn and a couple of estimators, notably logistic regression and K-nearest neighbors.

In this session we will take a look at model selection, one of the most important aspects of applied machine learning.

In [ ]:
# some imports

# plotting, set up plotly offline
from plotly.offline import init_notebook_mode, iplot, download_plotlyjs
init_notebook_mode()
import plotly.graph_objs as go

# numpy
import numpy as np

## Overfitting, Underfitting and Model Selection


The issues associated with validation and 
cross-validation are some of the most important
aspects of the practice of machine learning.  Selecting the optimal model
for your data is vital, and is a piece of the problem that is not often
appreciated by machine learning practitioners.

Of core importance is the following question:

**If our estimator is underperforming, how should we move forward?**

- Use simpler or more complicated model?
- Add more training samples?

The answer is often counter-intuitive.  In particular, **Sometimes using a
more complicated model will give _worse_ results.**  Also, **Sometimes adding
training data will not improve your results.**  The ability to determine
what steps will improve your model is what separates the successful machine
learning practitioners from the unsuccessful.

### Illustration of the Bias-Variance Tradeoff

For this section, we'll work with a simple 1D regression problem.  This will help us to
easily visualize the data and the model, and the results generalize easily to  higher-dimensional
datasets.  We'll explore a simple **linear regression** problem.
This can be accomplished within scikit-learn with the `sklearn.linear_model` module.

We'll create a simple nonlinear function that we'd like to fit

In [ ]:
def test_func(x, err=0.5):
    y = 10 - 1. / (x + 0.1)
    if err > 0:
        y = np.random.normal(y, err)
    return y

Now let's create a realization of this dataset:

In [ ]:
def make_data(N=40, error=1.0, random_seed=1):
    # randomly sample the data
    np.random.seed(1)
    X = np.random.random(N)[:, np.newaxis]
    y = test_func(X.ravel(), error)
    
    return X, y

In [ ]:
X, y = make_data(40, error=1)
iplot([go.Scatter(x=X.ravel(), y=y, mode='markers')])


Now say we want to perform a regression on this data.  Let's use the built-in linear regression function to compute a fit:

In [ ]:
X_test = np.linspace(-0.1, 1.1, 500)[:, None]

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
model = LinearRegression()
model.fit(X, y)
y_test = model.predict(X_test)

print("mean squared error: {0:.3g}".format(mean_squared_error(model.predict(X), y)))
iplot([go.Scatter(x=X.ravel(), y=y, mode='markers'),
        go.Scatter(x=X_test.ravel(), y=y_test, name='estimated model')])


We have fit a straight line to the data, but clearly this model is not a good choice.  We say that this model is **biased**, or that it **under-fits** the data.

Let's try to improve this by creating a more complicated model.  We can do this by adding degrees of freedom, and computing a polynomial regression over the inputs. Scikit-learn makes this easy with the ``PolynomialFeatures`` preprocessor, which can be pipelined with a linear regression.

Let's make a convenience routine to do this:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

Now we'll use this to fit a quadratic curve to the data.

In [ ]:
model = PolynomialRegression(2)
model.fit(X, y)
y_test = model.predict(X_test)


print("mean squared error: {0:.3g}".format(mean_squared_error(model.predict(X), y)))

iplot([go.Scatter(x=X.ravel(), y=y, mode='markers'),
        go.Scatter(x=X_test.ravel(), y=y_test, name='estimated model')])


This reduces the mean squared error, and makes a much better fit.  What happens if we use an even higher-degree polynomial?

In [ ]:
model = PolynomialRegression(10)
model.fit(X, y)
y_test = model.predict(X_test)

print("mean squared error: {0:.3g}".format(mean_squared_error(model.predict(X), y)))


layout = go.Layout(
#     yaxis=dict(
#         range=[0, 20]
#     )
)
data = [go.Scatter(x=X.ravel(), y=y, mode='markers', name='original data'),
        go.Scatter(x=X_test.ravel(), y=y_test, name='estimated model')]

fig = go.Figure(data=data, layout=layout)
iplot(fig)


When we increase the degree to this extent, it's clear that the resulting fit is no longer reflecting the true underlying distribution, but is more sensitive to the noise in the training data. For this reason, we call it a **high-variance model**, and we say that it **over-fits** the data.

### Detecting Over-fitting with Validation Curves

Clearly, computing the error on the training data is not enough (we saw this previously). As above, we can use **cross-validation** to get a better handle on how the model fit is working.

Let's do this here, again using the ``validation_curve`` utility. To make things more clear, we'll use a slightly larger dataset:

In [ ]:
X, y = make_data(120, error=1.0)
iplot([go.Scatter(x=X.ravel(), y=y, mode='markers')])

In [ ]:
from sklearn.learning_curve import validation_curve

degree = np.arange(0, 15)
val_train, val_test = validation_curve(PolynomialRegression(), X, y,
                                       'polynomialfeatures__degree', degree, 
                                       cv=7)

In [ ]:
val_train.mean(1)

Now let's plot the validation curves:

In [ ]:
def plot_learning_curves(val_train, val_test):
    data = [
        go.Scatter(
            x=degree,
            y=val_train.mean(1),
            error_y=dict(
                type='data',
                array=val_train.std(1),
                visible=True
            ),
            name='train error'
        ),
        go.Scatter(
            x=degree,
            y=val_test.mean(1),
            error_y=dict(
                type='data',
                array=val_train.std(1),
                visible=True
            ),
            name='validation error'
        )
    ]

    layout = go.Layout(
        xaxis=dict(
            title='Polynomial degree',
        ),
        yaxis=dict(
            title='Score',
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

plot_learning_curves(val_train, val_test)

Notice the trend here, which is common for this type of plot.

1. For a small model complexity, the training error and validation error are very similar. This indicates that the model is **under-fitting** the data: it doesn't have enough complexity to represent the data. Another way of putting it is that this is a **high-bias** model.

2. As the model complexity grows, the training and validation scores diverge. This indicates that the model is **over-fitting** the data: it has so much flexibility, that it fits the noise rather than the underlying trend. Another way of putting it is that this is a **high-variance** model.

3. Note that the training score (nearly) always improves with model complexity. This is because a more complicated model can fit the noise better, so the model improves. The validation data generally has a sweet spot, which here is around 5 terms.

# <span style='color: red'>__Tip__</span>: in the final report you __absolutely__ must include these plots.

Here's our best-fit model according to the cross-validation:

In [ ]:
model = PolynomialRegression(4).fit(X, y)

iplot([go.Scatter(x=X.ravel(), y=y, mode='markers'),
        go.Scatter(x=X_test.ravel(), y=model.predict(X_test), name='estimated model')])


### Exercise (10')

On the digits dataset, plot the validation curve for the ```C``` parameter in ```LogisticRegression```, with 10 values between $10^-6$ and $10^6$ in log scale.

In [ ]:
# setup code for exercise
from sklearn import datasets
from sklearn.model_selection import train_test_split

digits = datasets.load_digits()
X_digits = digits.data
y_digits = digits.target



---

### Automatically choose the best hyperparameter

We can use the above curves to choose the best hyperparameters for our model. We can simply iterate through the validation curve and select the best hyperparameter. However, there is a better way in scikit-learn: through the ```GridSearchCV``` object.

This is an object that, given data, computes the score during the fit of an estimator on a parameter grid and chooses the parameters to maximize the cross-validation score. This object takes an estimator during the construction and exposes an estimator API:


In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score

# get some data
digits = datasets.load_digits()
X = digits.data
y = digits.target

# define the different hyperparameters
Cs = np.logspace(-6, -1, 10)
print(Cs)

In [ ]:
# call GridSearchCV with a LogisticRegression estimator, but choosing
# C out of all Cs
from sklearn.linear_model import LogisticRegression
clf = GridSearchCV(estimator=LogisticRegression(), param_grid=dict(C=Cs))
clf.fit(X, y)        

So under the hood, ```GridSearchCV``` has:

  * Tried out all values of ```C``` in Cs.
  * Selected by cross-validation the one with best score.

Once fitted, the ```GridSearchCV``` behaves like any other estimator. Like any other estimator it implements a .predict and .score method.

In [ ]:
clf.predict(X)

In [ ]:
clf.score(X, y)

```GridSearchCV``` is what we call a **meta-estimator**: it behaves like an estimator but it is in fact made up from one or several estimators. This is a powerful concept!

How to retrieve the optimal value of C that was selected by ```GridSearchCV```? ```GridSearchCV``` has a member ```clf.best_estimator_``` that returns which was the best estimator.

In [ ]:
clf.best_estimator_

### Exercise 1


**Multi-layer Perceptron (MLP)** is a supervised learning algorithm that learns
a function $f(\cdot): R^m \rightarrow R^o$ by training on a dataset,
where $m$ is the number of dimensions for input and $o$ is the
number of dimensions for output. Given a set of features $X = \{x_1, x_2, ..., x_m\}$
and a target $y$, it can learn a non-linear function approximator for either
classification or regression. It is different from logistic regression, in that
between the input and the output layer, there can be one or more non-linear
layers, called hidden layers. This represents one hidden layer MLP with scalar
output.

<img src="http://scikit-learn.org/stable/_images/multilayerperceptron_network.png" width=400/>


On the digits dataset, build a neural network model, [MLPCLassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier) and:
  * choose the best possible parameters of ```alpha``` using ```GridSearchCV```.
  * choose the best possible parameter for ```hidden_layer_sizes```.

---

## Pipelines or combining different classifiers

At some point, you might want to combine some preprocessing (like feature selection) with a classifier. It is possible to do this manually:

In [ ]:
# get some data
from sklearn import datasets
digits = datasets.load_digits()
X = digits.data
y = digits.target


from sklearn import decomposition
# select only 10 features
pca = decomposition.PCA(n_components=10)
X_trans = pca.fit_transform(X)


# now train the classifier
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_trans, y)

In [ ]:
X_trans.shape

In [ ]:
X.shape

But this soon becomes tedious when dealing with cross-validation. A better solution is to create a Pipeline:

In [ ]:
from sklearn.pipeline import make_pipeline
clf = make_pipeline(
    decomposition.PCA(n_components=10), LogisticRegression())
clf.fit(X, y)
print(clf.score(X, y))

### Exercise

Create a Pipeline with PCA and LogisticRegression, plot performance as the number of dimensions (```n_components```) by cross-validation and get the optimum using GridSearchCV.

## Summary

We've gone over several useful tools for model validation

- The **Training Score** shows how well a model fits the data it was trained on. This is not a good indication of model effectiveness
- The **Validation Score** shows how well a model fits hold-out data. The most effective method is some form of cross-validation, where multiple hold-out sets are used.
- **Validation Curves** are a plot of validation score and training score as a function of **model complexity**:
  + when the two curves are close, it indicates *underfitting*
  + when the two curves are separated, it indicates *overfitting*
  + the "sweet spot" is in the middle
- **GridSearchCV** is an object that encapsulates the logic of performing model selection.
  + By specifying the different values of the parameter, he will try them out and keep the best.
  + The resulting object is a 
  
These tools are powerful means of evaluating your model on your data.

<small><i>This notebook was put together by Fabian Pedregosa based on work by [Jake Vanderplas](http://www.vanderplas.com).</i></small>